In [11]:
import pandas as pd
import re
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [2]:
# import data
list_data = []

# data = pd.read_csv('scrapegabungan-translated.csv', sep=";", encoding='unicode_escape')
data = pd.read_csv('LABELLED_DATA.csv', sep=";", encoding='unicode_escape')

In [3]:
list_data = data['REVIEW'].tolist()

In [6]:

temp_list = []
for data in list_data:
    temp = re.sub(r"^.*?Original.\s", '', data)
    temp = temp.encode('ascii', 'ignore').decode('ascii')
    temp_list.append(temp)
list_data = temp_list
df_list_data = pd.DataFrame(list_data)

In [7]:
list_data = df_list_data[0].tolist()

## cleaning data

In [9]:
#case folding
lower_sentences = []
for review in list_data:
    text = review.lower()
    lower_sentences.append(text)


['kondisi di dalam studio cukup bersih. ada 2 studio, jadi hanya bisa memutar 2 film sekaligus. dan kita bisa cek jadwal tayangnya di akun instagramnya, tapi hanya tersedia untuk film besok. kondisi kamar mandi dan musholla harus ditingkatkan kebersihannya, adapun masih cukup kotor dan berbau tidak sedap.',
 'fasilitas cukup untuk kategori nsc. tidak adanya layanan pembelian tiket online membuat antrian semakin panjang dan tidak efisien bagi masyarakat yang ingin membeli tiket.',
 'harga tiket yang sebenarnya murah malah lebih mahal karena harus beli minuman juga. dapat disimpulkan bahwa harganya cukup mahal, berbanding terbalik dengan kualitas studio.',
 'ini adalah satu-satunya bioskop teater di kota pasuruan. fasilitasnya biasa saja, tapi tidak terlalu bagus. sebenarnya kualitas suaranya di bawah rata-rata dibandingkan dengan bioskop lain di kota besar seperti surabaya atau malang.',
 'tapi ternyata tiketnya lebih murah.',
 'apa yang anda bayar adalah apa yang anda dapatkan. kursi m

In [7]:
# remove punctuation
no_punctuation = []
for review in lower_sentences:
    text = re.sub(r'[^\w\s]',' ',review)
    text = re.sub(r'\d','',text)
    text = re.sub(r'\s+',' ',text)
    no_punctuation.append(text)

In [8]:
# remove stopwords

factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()
remove_from_stopword = ['tidak', 'nggak', 'belum']
list_stopword = factory.get_stop_words()
temp = []
no_stopword = []

for word in list_stopword:
    if word not in remove_from_stopword:
        temp.append(word)
list_stopword = temp

i = 0
for text in no_punctuation:
    temp = ""
    text = text.split(" ")
    for review in text:
        if review not in list_stopword:
            temp = temp+" "+review
            temp = temp.strip()
    no_stopword.append(temp)
    i+=1

In [9]:
# stemming
factory = StemmerFactory()
stemmer = factory.create_stemmer()

clean_word = []

for word in no_stopword:
    hasil = stemmer.stem(word)
    clean_word.append(hasil)

In [10]:
splitted_word = {}
for review in clean_word:
    alphanumerical_word = re.sub(r'[^\w\s]',' ',review)
    raw_review = alphanumerical_word.strip()
    words = raw_review.split(" ")
    for word in words:
        if word not in splitted_word:
            splitted_word[word] = 0
        splitted_word[word]+=1


splitted_word
df_word = pd.DataFrame(data=splitted_word, index=[0])
# df_word.to_excel("freqword.xlsx", index=False)
# df_word.to_excel("freqword_LABELLED.xlsx", index=False)

In [11]:
# words_corpus = pd.read_excel('freqword.xlsx', 'data_fix')
words_corpus = pd.read_excel('freqword_LABELLED.xlsx', 'data_fix')
result = []
normalizad_word_dict = {}

for index, row in words_corpus.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1]

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

normalized = []
result_joined = ""
for text in clean_word:
    temp = []
    text = text.split(" ")
    result = normalized_term(text)
    result_joined = ' '.join(map(str, result))
    normalized.append(result_joined)

In [12]:
# remove stopwords & outliers

factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()
remove_from_stopword = ['tidak','enggak', 'belum']
df_add_stopwords = pd.read_csv('list_tambahan_stopword.txt', sep=" ")
add_stopwords = df_add_stopwords['list'].tolist()
list_stopword = factory.get_stop_words()
temp = []
no_stopword_outlier = []

list_stopword.extend(add_stopwords)
for word in list_stopword:
    if word not in remove_from_stopword:
        temp.append(word)
list_stopword = temp

i = 0
for text in normalized:
    temp = ""
    text = text.split(" ")
    for review in text:
        if review not in list_stopword:
            temp = temp+" "+review
            temp = temp.strip()
    no_stopword_outlier.append(temp)
    i+=1

In [13]:
merged_list = {'RAW DATA' : list_data ,
'CASE FOLDING' : lower_sentences,
'REMOVE PUNCTUATION' : no_punctuation, 
'REMOVE STOPWORD' : no_stopword, 
'STEMMING' : clean_word,
'NORMALIZATION' : normalized,
'NO OUTLIER & STOPWORD' : no_stopword_outlier}
list_dataframe = pd.DataFrame(merged_list)
list_dataframe

,RAW DATA,CASE FOLDING,REMOVE PUNCTUATION,REMOVE STOPWORD,STEMMING,NORMALIZATION,NO OUTLIER & STOPWORD
0,Kondisi di dalam studio cukup bersih. Ada 2 st...,kondisi di dalam studio cukup bersih. ada 2 st...,kondisi di dalam studio cukup bersih ada studi...,kondisi studio cukup bersih studio jadi memuta...,kondisi studio cukup bersih studio jadi putar ...,kondisi studio cukup bersih studio jadi putar ...,kondisi studio cukup bersih studio jadi putar ...
1,Fasilitas cukup untuk kategori NSC. Tidak adan...,fasilitas cukup untuk kategori nsc. tidak adan...,fasilitas cukup untuk kategori nsc tidak adany...,fasilitas cukup kategori nsc tidak adanya laya...,fasilitas cukup kategori nsc tidak ada layan b...,fasilitas cukup kategori new star cineplex tid...,fasilitas cukup kategori new star cineplex tid...
2,harga tiket yang sebenarnya murah malah lebih ...,harga tiket yang sebenarnya murah malah lebih ...,harga tiket yang sebenarnya murah malah lebih ...,harga tiket sebenarnya murah malah lebih mahal...,harga tiket benar murah malah lebih mahal beli...,harga tiket benar murah malah lebih mahal beli...,harga tiket benar murah malah lebih mahal beli...
3,Ini adalah satu-satunya bioskop teater di kota...,ini adalah satu-satunya bioskop teater di kota...,ini adalah satu satunya bioskop teater di kota...,satu satunya bioskop teater kota pasuruan fasi...,satu satu bioskop teater kota pasuruan fasilit...,satu satu bioskop teater kota pasuruan fasilit...,satu satu bioskop teater kota pasuruan fasilit...
4,Tapi ternyata tiketnya lebih murah.,tapi ternyata tiketnya lebih murah.,tapi ternyata tiketnya lebih murah,ternyata tiketnya lebih murah,nyata tiket lebih murah,nyata tiket lebih murah,nyata tiket lebih murah
...,...,...,...,...,...,...,...
791,"studionya perlu ditambah, banyak film box offi...","studionya perlu ditambah, banyak film box offi...",studionya perlu ditambah banyak film box offic...,studionya perlu ditambah banyak film box offic...,studio perlu tambah banyak film box office tid...,studio perlu tambah banyak film kotak kantor t...,studio perlu tambah banyak film kotak kantor t...
792,Paimin gk pernah ngerespon,paimin gk pernah ngerespon,paimin gk pernah ngerespon,paimin gk pernah ngerespon,paimin gk pernah ngerespon,admin gak pernah respon,admin gak pernah respon
793,Kursi banyak yg rusak,kursi banyak yg rusak,kursi banyak yg rusak,kursi banyak yg rusak,kursi banyak yg rusak,kursi banyak yang rusak,kursi banyak rusak
794,"Pemberitahuannya kurang update, trus film nya ...","pemberitahuannya kurang update, trus film nya ...",pemberitahuannya kurang update trus film nya j...,pemberitahuannya kurang update trus film nya s...,pemberitahuannya kurang update trus film nya s...,beri tahu kurang baru terus film nya sering fi...,beri tahu kurang baru terus film sering film luar
